In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from binance.client import Client
from xgboost import XGBRegressor
import pandas as pd
import pandas as pd
import hopsworks
import os
import warnings
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
warnings.filterwarnings("ignore")

In [111]:
os.environ["HOPSWORKS_API_KEY"] = "WMg6iOTOV75YfUcb.AcuNGWjcMS79s4bz0FqZsRaJtcU8Siqu7Mx084dfkRHauMxK251VTYLYSBAIqBb5"
project = hopsworks.login()
fs = project.get_feature_store() 
fg = fs.get_feature_group('data_historic_shifted', version=1)
df = fg.read()

2025-01-05 19:42:44,532 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 19:42:44,538 INFO: Initializing external client
2025-01-05 19:42:44,538 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 19:42:45,878 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1168537
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.27s) 


In [112]:
df.head()

,timestamp,inflation_rate,open,high,low,gold_price,volume
0,2024-07-01 01:00:00,0.074474,2313.0,2313.0,2308.0,2317.0,8.2880
1,2024-03-14 12:00:00,-0.009152,2148.0,2150.0,2146.0,2147.0,54.5260
2,2024-05-14 13:00:00,0.129762,2332.0,2334.0,2325.0,2337.0,48.4734
3,2024-10-03 19:00:00,-0.081596,2665.0,2669.0,2664.0,2664.0,32.2692
4,2024-10-27 16:00:00,0.009263,2761.0,2767.0,2761.0,2767.0,20.1244


In [113]:
features = fg.select_all()
features.show(5)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.78s) 


,timestamp,inflation_rate,open,high,low,gold_price,volume
0,2024-07-01 01:00:00,0.074474,2313.0,2313.0,2308.0,2317.0,8.2880
1,2024-03-14 12:00:00,-0.009152,2148.0,2150.0,2146.0,2147.0,54.5260
2,2024-05-14 13:00:00,0.129762,2332.0,2334.0,2325.0,2337.0,48.4734
3,2024-10-03 19:00:00,-0.081596,2665.0,2669.0,2664.0,2664.0,32.2692
4,2024-10-27 16:00:00,0.009263,2761.0,2767.0,2761.0,2767.0,20.1244


In [114]:
feature_view = fs.get_or_create_feature_view(
    name='gold_prices_fv',
    description="gold_price",
    version=1,
    labels=['gold_price'],
    query=features,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1168537/fs/1159240/fv/gold_prices_fv/version/1


In [115]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.25s) 


In [116]:
features_t = X_train.drop(['timestamp'], axis=1)
features_test = X_test.drop(['timestamp'], axis=1)

In [117]:
my_model = XGBRegressor()
my_model.fit(features_t, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [118]:
pred = my_model.predict(features_test)
MSE = mean_squared_error(y_test, pred)
print("The MSE is :", MSE)


The MSE is : 43.113203780853176


In [119]:
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
schema_to_dict = model_schema.to_dict()
my_model.save_model('gold_model' + "/model.json")

In [120]:
proj = project.get_model_registry()
g_p_model = proj.python.create_model(
    name="gold_final_final", 
    model_schema=model_schema,
    input_example=X_test.sample().values, 
    description="Gold price prediction hopworks",
)
g_p_model.save("gold_model")

Uploading: 100.000%|██████████| 525767/525767 elapsed<00:02 remaining<00:00  1.20it/s]
Uploading: 100.000%|██████████| 62/62 elapsed<00:02 remaining<00:0003<00:07,  1.92s/it]
Uploading: 100.000%|██████████| 603/603 elapsed<00:01 remaining<00:00
Model export complete: 100%|██████████| 6/6 [00:13<00:00,  2.23s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/1168537/models/gold_final_final/1


Model(name: 'gold_final_final', version: 1)